# Exercício 02 - Comparação de Redes Neurais Rasas

Aluno: Frederico Luis de Azevedo

Professor: Dr. Francisco de Assis Boldt

## Bibliotecas e Inicialização

In [13]:
from sklearn.datasets import make_classification

import numpy as np

In [3]:
X, y = make_classification(n_samples=100000, n_features=20,
                                    n_informative=2, n_redundant=2)

In [7]:
X.shape

(100000, 20)

In [14]:
unique, counts = np.unique(y, return_counts=True)
>>> dict(zip(unique, counts))

{0: 50003, 1: 49997}